In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [86]:
# Load the dataset
df = pd.read_csv('..//..//dataset//preliminary_dataset.csv')  


# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [87]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(80000, 1, 6) (80000,) (100000, 1, 6)


# Model

In [88]:
def build_drnn_lstm():
    model = Sequential()
    
    # Input layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    
    # DRNN layer
    model.add(SimpleRNN(units=100, activation='relu', return_sequences=True))
    model.add(SimpleRNN(units=100, activation='relu', return_sequences=True))
    
    # LSTM layer
    model.add(LSTM(units=100,activation='relu', return_sequences=True))
    model.add(LSTM(units=100, activation='relu',return_sequences=True))
    model.add(LSTM(units=100, ))
    
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])
    
    return model


In [89]:
# Instantiate and train the model
model = build_drnn_lstm()
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - loss: 0.2863 - mae: 0.2056 - mse: 0.0982 - val_loss: 0.0012 - val_mae: 0.0012 - val_mse: 4.9939e-05
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 8.9107e-04 - mae: 8.0717e-04 - mse: 5.4255e-05 - val_loss: 2.0578e-04 - val_mae: 2.0525e-04 - val_mse: 1.0190e-06
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - loss: 3.7260e-04 - mae: 2.9640e-04 - mse: 4.5996e-05 - val_loss: 9.3911e-05 - val_mae: 9.3770e-05 - val_mse: 2.7388e-07
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.0013 - mae: 6.7222e-04 - mse: 2.2942e-04 - val_loss: 1.1144e-04 - val_mae: 8.0651e-05 - val_mse: 3.0364e-05
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - loss: 5.9319e-04 - mae: 2.9301e-04 - mse: 7.9381e-05 - val_loss: 0.0572 - val_mae: 0.0305 - val_mse: 0.0198
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.1365 - mae: 0.0611 - mse: 0.0308 - val_loss: 1.0118e-04 - val_mae: 9.9811e-05 - val_mse: 2.5372

In [90]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [91]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

Mean Squared Error (MSE): 0.0000001144398212
Root Mean Squared Error (RMSE): 0.0003382895522699


In [92]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()


In [93]:
import time

# Start time
start_time = time.time()

# Make predictions
y_pred = model.predict(X_test)

# End time
end_time = time.time()

# Total AES time
total_time = end_time - start_time  

# AES per sample
aes_per_sample = (total_time / len(X_test)) * 1e6  # Convert to microseconds (µs)

print(f'Total AES Time: {total_time:.6f} seconds')
print(f'AES Time per Sample: {aes_per_sample:.2f} µs/sample')


625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Total AES Time: 2.085996 seconds
AES Time per Sample: 104.30 µs/sample
